In [5]:
import numpy as np
from Bio import SeqIO
import gzip
import os
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, Dropout
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import pandas as pd

np.random.seed(454)


def load_data(path):
    data = gzip.open(os.path.join(path,"sequences.fa.gz"),"rt")
    return data


def get_seq(protein, t_data):
    
    training_data = load_data("data/clip/%s/5000/training_sample_0"% protein)
    x_train = np.zeros((5000,101,4))
    r = 0    

    for record in SeqIO.parse(training_data,"fasta"):
        sequence = list(record.seq)                
        nucleotide = {'A' : 0, 'T' : 1, 'G' : 2, 'C' : 3, 'N' : 4} 
        num_seq = list() #sekvenca v številskem formatu


        for i in range(0,len(sequence)):
                num_seq.append(nucleotide[sequence[i]])


        X = np.zeros((1,len(num_seq),4))

        for i in range (len(num_seq)):
                if num_seq[i] <= 3:
                    X[:,i,num_seq[i]] = 1               

        x_train[r,:,:] = X
        r = r + 1
    
    return x_train


def get_class(protein, t_data):
    y_train = []
    

    if t_data == 'train':
        data = load_data("data/clip/%s/5000/training_sample_0"% protein)

    elif t_data == 'test':
        data = load_data("data/clip/%s/5000/test_sample_0"% protein)


    for record in SeqIO.parse(data,"fasta"):
        v = int((record.description).split(":")[1])
        y_train.append([int(v == 0), int(v != 0)])

    y_train = np.array(y_train)
    return y_train


def get_cobinding(protein, t_data):
    with gzip.open(("data/clip/%s/5000/training_sample_0/matrix_Cobinding.tab.gz"% protein), "rt") as f:
        cobinding_data = np.loadtxt(f, skiprows=1) 
    
    
    cobinding = np.zeros((5000,101,cobinding_data.shape[1]/101),dtype=np.int) #ustvarim prazen array


    for n in range(0,cobinding_data.shape[1],101):
        a = cobinding_data[:,n:(n+101)]
        cobinding[:,:,(n/101)] = a
    
    return cobinding
    

def get_region (protein, t_data):

    with gzip.open(("data/clip/%s/5000/training_sample_0/matrix_RegionType.tab.gz"% protein), "rt") as f:
        region_data = np.loadtxt(f, skiprows=1) 


    region = np.zeros((5000,101,region_data.shape[1]/101),dtype=np.int) #ustvarim prazen array


    for n in range(0,region_data.shape[1],101):
        a = region_data[:,n:(n+101)]
        region[:,:,(n/101)] = a
        
    return region


In [6]:
score_list = []
protein_list = ["1_PARCLIP_AGO1234_hg19", "2_PARCLIP_AGO2MNASE_hg19","3_HITSCLIP_Ago2_binding_clusters","4_HITSCLIP_Ago2_binding_clusters_2","5_CLIPSEQ_AGO2_hg19", "6_CLIP-seq-eIF4AIII_1","7_CLIP-seq-eIF4AIII_2","8_PARCLIP_ELAVL1_hg19","9_PARCLIP_ELAVL1MNASE_hg19", "10_PARCLIP_ELAVL1A_hg19", "10_PARCLIP_ELAVL1A_hg19", "12_PARCLIP_EWSR1_hg19", "13_PARCLIP_FUS_hg19", "14_PARCLIP_FUS_mut_hg19", "15_PARCLIP_IGF2BP123_hg19", "16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters", "17_ICLIP_HNRNPC_hg19", "18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome", "19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome", "20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome", "21_PARCLIP_MOV10_Sievers_hg19", "22_ICLIP_NSUN2_293_group_4007_all-NSUN2-293-hg19_sum_G_hg19--ensembl59_from_3137-3202_bedGraph-cDNA-hits-in-genome", "23_PARCLIP_PUM2_hg19", "24_PARCLIP_QKI_hg19", "25_CLIPSEQ_SFRS1_hg19","26_PARCLIP_TAF15_hg19", "27_ICLIP_TDP43_hg19", "28_ICLIP_TIA1_hg19", "29_ICLIP_TIAL1_hg19", "30_ICLIP_U2AF65_Hela_iCLIP_ctrl_all_clusters", "31_ICLIP_U2AF65_Hela_iCLIP_ctrl+kd_all_clusters"]

for x in range (0,31):

    protein = protein_list[x]
    
    X = np.dstack((get_cobinding(protein,"train"), get_seq(protein,"train"), get_region(protein,"train")))
    y = get_class(protein,"train")
    
    size = X.shape[2]
    
    kf = KFold(n_splits=10) #10 različnih delitev podatkov  
    
    score = []
    for train_index, test_index in kf.split(X):
        
        # sestavljanje modela
        model = Sequential()
        model.add(Conv1D(120, 24,data_format='channels_last', input_shape=(101, size) , strides = 1, padding='valid'))
        model.add(MaxPooling1D(pool_size=24, strides=6, padding='valid'))
        model.add(Conv1D(120, 4, activation='relu'))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(200, activation='relu'))
        model.add(Dense(2, activation='sigmoid'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

          
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)
   
    

        y_scores =(model.predict(X_test))# pridobim napovedi za sekvence iz cobinding
        score.append(roc_auc_score(y_test, y_scores))
        
    
    
    print ("%s finishhed" % (protein))
    score_list.append(np.mean(score))
    
    
print(score_list)



# Table for comparing results
ionmf = [0.913, 0.714, 0.928, 0.925, 0.752, 0.939, 0.942, 0.954, 0.723, 0.974, 0.971, 0.893, 0.811, 0.955, 0.936, 0.942, 0.976, 0.826, 0.801, 0.796, 0.956, 0.868, 0.961, 0.877, 0.898, 0.910, 0.860, 0.943, 0.942, 0.915, 0.925]
difference = np.subtract(score_list,ionmf)
pd.set_option('display.max_columns', 4)


              
table_data = {'Score' : score_list, 'iONMF':ionmf, 'Difference':difference}
table = pd.DataFrame(table_data, index = protein_list)

print (table)


1_PARCLIP_AGO1234_hg19 finishhed
2_PARCLIP_AGO2MNASE_hg19 finishhed
3_HITSCLIP_Ago2_binding_clusters finishhed
4_HITSCLIP_Ago2_binding_clusters_2 finishhed
5_CLIPSEQ_AGO2_hg19 finishhed
6_CLIP-seq-eIF4AIII_1 finishhed
7_CLIP-seq-eIF4AIII_2 finishhed
8_PARCLIP_ELAVL1_hg19 finishhed
9_PARCLIP_ELAVL1MNASE_hg19 finishhed
10_PARCLIP_ELAVL1A_hg19 finishhed
10_PARCLIP_ELAVL1A_hg19 finishhed
12_PARCLIP_EWSR1_hg19 finishhed
13_PARCLIP_FUS_hg19 finishhed
14_PARCLIP_FUS_mut_hg19 finishhed
15_PARCLIP_IGF2BP123_hg19 finishhed
16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters finishhed
17_ICLIP_HNRNPC_hg19 finishhed
18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome finishhed
19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome finishhed
20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome finishhed
21_PARC

In [ ]:
score_list = []
protein_list = ["1_PARCLIP_AGO1234_hg19", "2_PARCLIP_AGO2MNASE_hg19","3_HITSCLIP_Ago2_binding_clusters","4_HITSCLIP_Ago2_binding_clusters_2","5_CLIPSEQ_AGO2_hg19", "6_CLIP-seq-eIF4AIII_1","7_CLIP-seq-eIF4AIII_2","8_PARCLIP_ELAVL1_hg19","9_PARCLIP_ELAVL1MNASE_hg19", "10_PARCLIP_ELAVL1A_hg19", "10_PARCLIP_ELAVL1A_hg19", "12_PARCLIP_EWSR1_hg19", "13_PARCLIP_FUS_hg19", "14_PARCLIP_FUS_mut_hg19", "15_PARCLIP_IGF2BP123_hg19", "16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters", "17_ICLIP_HNRNPC_hg19", "18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome", "19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome", "20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome", "21_PARCLIP_MOV10_Sievers_hg19", "22_ICLIP_NSUN2_293_group_4007_all-NSUN2-293-hg19_sum_G_hg19--ensembl59_from_3137-3202_bedGraph-cDNA-hits-in-genome", "23_PARCLIP_PUM2_hg19", "24_PARCLIP_QKI_hg19", "25_CLIPSEQ_SFRS1_hg19","26_PARCLIP_TAF15_hg19", "27_ICLIP_TDP43_hg19", "28_ICLIP_TIA1_hg19", "29_ICLIP_TIAL1_hg19", "30_ICLIP_U2AF65_Hela_iCLIP_ctrl_all_clusters", "31_ICLIP_U2AF65_Hela_iCLIP_ctrl+kd_all_clusters"]

for x in range (0,31):

    protein = protein_list[x]
    
    X = np.dstack((get_cobinding(protein,"train"), get_seq(protein,"train"), get_region(protein,"train")))
    y = get_class(protein,"train")
    
    size = X.shape[2]
    
    kf = KFold(n_splits=10) #10 različnih delitev podatkov  
    
    score = []
    for train_index, test_index in kf.split(X):
        
        # sestavljanje modela
        model = Sequential()
        model.add(Conv1D(120, 24,data_format='channels_last', input_shape=(101, size) , strides = 1, padding='valid'))
        model.add(MaxPooling1D(pool_size=12, strides=2, padding='valid'))
        model.add(Conv1D(120, 4, activation='relu'))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(200, activation='relu'))
        model.add(Dense(2, activation='sigmoid'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

          
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train, y_train, epochs=10, batch_size=16, verbose=0)
   
    

        y_scores =(model.predict(X_test))# pridobim napovedi za sekvence iz cobinding
        score.append(roc_auc_score(y_test, y_scores))
        
    
    
    print ("%s finishhed" % (protein))
    score_list.append(np.mean(score))
    
    
print(score_list)



# Table for comparing results
ionmf = [0.913, 0.714, 0.928, 0.925, 0.752, 0.939, 0.942, 0.954, 0.723, 0.974, 0.971, 0.893, 0.811, 0.955, 0.936, 0.942, 0.976, 0.826, 0.801, 0.796, 0.956, 0.868, 0.961, 0.877, 0.898, 0.910, 0.860, 0.943, 0.942, 0.915, 0.925]
difference = np.subtract(score_list,ionmf)
pd.set_option('display.max_columns', 4)


              
table_data = {'Score' : score_list, 'iONMF':ionmf, 'Difference':difference}
table = pd.DataFrame(table_data, index = protein_list)

print (table)


In [8]:
import numpy as np
from Bio import SeqIO
import gzip
import os
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, MaxPooling1D, Flatten, GlobalAveragePooling1D, Dropout
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import pandas as pd

protein_list = ["1_PARCLIP_AGO1234_hg19", "2_PARCLIP_AGO2MNASE_hg19","3_HITSCLIP_Ago2_binding_clusters","4_HITSCLIP_Ago2_binding_clusters_2","5_CLIPSEQ_AGO2_hg19", "6_CLIP-seq-eIF4AIII_1","7_CLIP-seq-eIF4AIII_2","8_PARCLIP_ELAVL1_hg19","9_PARCLIP_ELAVL1MNASE_hg19", "10_PARCLIP_ELAVL1A_hg19", "10_PARCLIP_ELAVL1A_hg19", "12_PARCLIP_EWSR1_hg19", "13_PARCLIP_FUS_hg19", "14_PARCLIP_FUS_mut_hg19", "15_PARCLIP_IGF2BP123_hg19", "16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters", "17_ICLIP_HNRNPC_hg19", "18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome", "19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome", "20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome", "21_PARCLIP_MOV10_Sievers_hg19", "22_ICLIP_NSUN2_293_group_4007_all-NSUN2-293-hg19_sum_G_hg19--ensembl59_from_3137-3202_bedGraph-cDNA-hits-in-genome", "23_PARCLIP_PUM2_hg19", "24_PARCLIP_QKI_hg19", "25_CLIPSEQ_SFRS1_hg19","26_PARCLIP_TAF15_hg19", "27_ICLIP_TDP43_hg19", "28_ICLIP_TIA1_hg19", "29_ICLIP_TIAL1_hg19", "30_ICLIP_U2AF65_Hela_iCLIP_ctrl_all_clusters", "31_ICLIP_U2AF65_Hela_iCLIP_ctrl+kd_all_clusters"]
score_list= [0.9166314660608545, 0.7355842739040614, 0.9156874064729882, 0.9110003605733115, 0.7106967639824682, 0.9175852831044603, 0.9532003478242389, 0.9609342519839978, 0.7175438955225013, 0.9260018057724553, 0.9309924438043105, 0.9323816311487496, 0.9071851247804773, 0.9831397242073813, 0.9433773325676775, 0.9346670168149185, 0.9593113189919608, 0.7925042050312222, 0.7068700113148783, 0.7681858296389412, 0.969496120005114, 0.812325268153552, 0.9779414360292946, 0.9469422043878664, 0.8703725801929352, 0.9702106147520017, 0.8781391345632541, 0.9387041550637198, 0.9376481211314986, 0.9335122789612825, 0.9341154218778888]
ionmf = [0.913, 0.714, 0.928, 0.925, 0.752, 0.939, 0.942, 0.954, 0.723, 0.974, 0.971, 0.893, 0.811, 0.955, 0.936, 0.942, 0.976, 0.826, 0.801, 0.796, 0.956, 0.868, 0.961, 0.877, 0.898, 0.910, 0.860, 0.943, 0.942, 0.915, 0.925]
difference = np.subtract(score_list,ionmf)
pd.set_option('display.max_columns', 4)


              
table_data = {'Score' : score_list, 'iONMF':ionmf, 'Difference':difference}
table = pd.DataFrame(table_data, index = protein_list)

print (table)



                                                    Difference     Score  \
1_PARCLIP_AGO1234_hg19                                0.003631  0.916631   
2_PARCLIP_AGO2MNASE_hg19                              0.021584  0.735584   
3_HITSCLIP_Ago2_binding_clusters                     -0.012313  0.915687   
4_HITSCLIP_Ago2_binding_clusters_2                   -0.014000  0.911000   
5_CLIPSEQ_AGO2_hg19                                  -0.041303  0.710697   
6_CLIP-seq-eIF4AIII_1                                -0.021415  0.917585   
7_CLIP-seq-eIF4AIII_2                                 0.011200  0.953200   
8_PARCLIP_ELAVL1_hg19                                 0.006934  0.960934   
9_PARCLIP_ELAVL1MNASE_hg19                           -0.005456  0.717544   
10_PARCLIP_ELAVL1A_hg19                              -0.047998  0.926002   
10_PARCLIP_ELAVL1A_hg19                              -0.040008  0.930992   
12_PARCLIP_EWSR1_hg19                                 0.039382  0.932382   
13_PARCLIP_F

In [2]:
'''
arhitektura modela 
 
model = Sequential()
        model.add(Conv1D(120,4,data_format='channels_last', input_shape=(101, size) , strides = 1, padding='valid'))
        model.add(MaxPooling1D(pool_size=24, strides=2, padding='valid'))
        model.add(Conv1D(120, 4, activation='relu'))
        model.add(GlobalAveragePooling1D())
        model.add(Dense(200, activation='relu'))
        model.add(Dense(2, activation='sigmoid'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

'''


import numpy as np
import pandas as pd
score_list = [0.91249816495929, 0.7626450772807127, 0.9084510844575933, 0.9196874015687442, 0.7334162319501186, 0.9173341246812802, 0.9554295878289565, 0.9643697034835925, 0.7548565940975945, 0.9301782951531139, 0.9308885650545436, 0.932926330425737, 0.898857053374814, 0.976241435704717, 0.9433553857366352, 0.9271558289215133, 0.94217081786666, 0.81116644993974, 0.765878379881905, 0.7998138612249177, 0.9745312431338725, 0.8509365580449263, 0.976889009288503, 0.945752770259, 0.8732536961307217, 0.9574785004610682, 0.8793866943231377, 0.9374699429100511, 0.9276089090165114, 0.91500985610617, 0.9234073665024163]
protein_list = ["1_PARCLIP_AGO1234_hg19", "2_PARCLIP_AGO2MNASE_hg19","3_HITSCLIP_Ago2_binding_clusters","4_HITSCLIP_Ago2_binding_clusters_2","5_CLIPSEQ_AGO2_hg19", "6_CLIP-seq-eIF4AIII_1","7_CLIP-seq-eIF4AIII_2","8_PARCLIP_ELAVL1_hg19","9_PARCLIP_ELAVL1MNASE_hg19", "10_PARCLIP_ELAVL1A_hg19", "10_PARCLIP_ELAVL1A_hg19", "12_PARCLIP_EWSR1_hg19", "13_PARCLIP_FUS_hg19", "14_PARCLIP_FUS_mut_hg19", "15_PARCLIP_IGF2BP123_hg19", "16_ICLIP_hnRNPC_Hela_iCLIP_all_clusters", "17_ICLIP_HNRNPC_hg19", "18_ICLIP_hnRNPL_Hela_group_3975_all-hnRNPL-Hela-hg19_sum_G_hg19--ensembl59_from_2337-2339-741_bedGraph-cDNA-hits-in-genome", "19_ICLIP_hnRNPL_U266_group_3986_all-hnRNPL-U266-hg19_sum_G_hg19--ensembl59_from_2485_bedGraph-cDNA-hits-in-genome", "20_ICLIP_hnRNPlike_U266_group_4000_all-hnRNPLlike-U266-hg19_sum_G_hg19--ensembl59_from_2342-2486_bedGraph-cDNA-hits-in-genome", "21_PARCLIP_MOV10_Sievers_hg19", "22_ICLIP_NSUN2_293_group_4007_all-NSUN2-293-hg19_sum_G_hg19--ensembl59_from_3137-3202_bedGraph-cDNA-hits-in-genome", "23_PARCLIP_PUM2_hg19", "24_PARCLIP_QKI_hg19", "25_CLIPSEQ_SFRS1_hg19","26_PARCLIP_TAF15_hg19", "27_ICLIP_TDP43_hg19", "28_ICLIP_TIA1_hg19", "29_ICLIP_TIAL1_hg19", "30_ICLIP_U2AF65_Hela_iCLIP_ctrl_all_clusters", "31_ICLIP_U2AF65_Hela_iCLIP_ctrl+kd_all_clusters"]
ionmf = [0.913, 0.714, 0.928, 0.925, 0.752, 0.939, 0.942, 0.954, 0.723, 0.974, 0.971, 0.893, 0.811, 0.955, 0.936, 0.942, 0.976, 0.826, 0.801, 0.796, 0.956, 0.868, 0.961, 0.877, 0.898, 0.910, 0.860, 0.943, 0.942, 0.915, 0.925]
difference = np.subtract(score_list,ionmf)
pd.set_option('display.max_columns', 4)


              
table_data = {'Score' : score_list, 'Difference':difference}
table = pd.DataFrame(table_data, index = protein_list)

print (table)



                                                    Difference     Score
1_PARCLIP_AGO1234_hg19                               -0.000502  0.912498
2_PARCLIP_AGO2MNASE_hg19                              0.048645  0.762645
3_HITSCLIP_Ago2_binding_clusters                     -0.019549  0.908451
4_HITSCLIP_Ago2_binding_clusters_2                   -0.005313  0.919687
5_CLIPSEQ_AGO2_hg19                                  -0.018584  0.733416
6_CLIP-seq-eIF4AIII_1                                -0.021666  0.917334
7_CLIP-seq-eIF4AIII_2                                 0.013430  0.955430
8_PARCLIP_ELAVL1_hg19                                 0.010370  0.964370
9_PARCLIP_ELAVL1MNASE_hg19                            0.031857  0.754857
10_PARCLIP_ELAVL1A_hg19                              -0.043822  0.930178
10_PARCLIP_ELAVL1A_hg19                              -0.040111  0.930889
12_PARCLIP_EWSR1_hg19                                 0.039926  0.932926
13_PARCLIP_FUS_hg19                                

In [4]:
score_list1= [0.9166314660608545, 0.7355842739040614, 0.9156874064729882, 0.9110003605733115, 0.7106967639824682, 0.9175852831044603, 0.9532003478242389, 0.9609342519839978, 0.7175438955225013, 0.9260018057724553, 0.9309924438043105, 0.9323816311487496, 0.9071851247804773, 0.9831397242073813, 0.9433773325676775, 0.9346670168149185, 0.9593113189919608, 0.7925042050312222, 0.7068700113148783, 0.7681858296389412, 0.969496120005114, 0.812325268153552, 0.9779414360292946, 0.9469422043878664, 0.8703725801929352, 0.9702106147520017, 0.8781391345632541, 0.9387041550637198, 0.9376481211314986, 0.9335122789612825, 0.9341154218778888]
score_list2 = [0.91249816495929, 0.7626450772807127, 0.9084510844575933, 0.9196874015687442, 0.7334162319501186, 0.9173341246812802, 0.9554295878289565, 0.9643697034835925, 0.7548565940975945, 0.9301782951531139, 0.9308885650545436, 0.932926330425737, 0.898857053374814, 0.976241435704717, 0.9433553857366352, 0.9271558289215133, 0.94217081786666, 0.81116644993974, 0.765878379881905, 0.7998138612249177, 0.9745312431338725, 0.8509365580449263, 0.976889009288503, 0.945752770259, 0.8732536961307217, 0.9574785004610682, 0.8793866943231377, 0.9374699429100511, 0.9276089090165114, 0.91500985610617, 0.9234073665024163]

np.subtract(score_list2, score_list1)


array([-4.13330110e-03,  2.70608034e-02, -7.23632202e-03,  8.68704100e-03,
        2.27194680e-02, -2.51158423e-04,  2.22924000e-03,  3.43545150e-03,
        3.73126986e-02,  4.17648938e-03, -1.03878750e-04,  5.44699277e-04,
       -8.32807141e-03, -6.89828850e-03, -2.19468310e-05, -7.51118789e-03,
       -1.71405011e-02,  1.86622449e-02,  5.90083686e-02,  3.16280316e-02,
        5.03512313e-03,  3.86112899e-02, -1.05242674e-03, -1.18943413e-03,
        2.88111594e-03, -1.27321143e-02,  1.24755976e-03, -1.23421215e-03,
       -1.00392121e-02, -1.85024229e-02, -1.07080554e-02])